Imports

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import numpy.linalg as la
import pickle
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.metrics import confusion_matrix
import scipy.spatial.distance as spd
from scipy.linalg import pinv

In [3]:
D1 = pickle.load(open('A47467_Q001_data.p', 'rb'))
D2 = pickle.load(open('A47467_Q002_data.p', 'rb'))

# Pergunta 1

In [4]:
X1 = D1['Xtrain']
X2 = D1['Xtest']

y1 = D1['trueClassTrain']
y2 = D1['trueClassTest']

print(X1.shape, y1.shape, X2.shape, y2.shape)

(2, 265) (265,) (2, 797) (797,)


In [5]:
def distancia(x, y):
    M = np.array([[1,-2],[1,1]])
    return np.dot((x-y).T, np.dot(M, (x-y)))

In [6]:
knn = KNeighborsClassifier(n_neighbors=29, metric=distancia).fit(X1.T, y1)
y2e = knn.predict(X2.T)
cm = confusion_matrix(y2, y2e)
print(cm)

[[180   3   0   0   6]
 [ 27 133   0   0   4]
 [  5   0 174   0   9]
 [  0   0   0 106   0]
 [ 10   3  10   0 127]]


In [7]:
print(cm[2][3])
print(cm[4][4])

0
127


In [8]:
print(cm[0][0])
print(cm[1][1])

180
133


# Pergunta 2

In [9]:
y = D2['trueClass']
ye = D2['estClass']
cm = confusion_matrix(y,ye)
cm

array([[786, 174],
       [ 60, 180]])

Accuracy = $\frac{TP + TN}{TP + TN + FP + FN}$

Precision (Positive Predictive Value) = $\frac{TP}{TP + FP}$

Recall (True Positive Rate) = $\frac{TP}{TP + FN}$ = 1 - FNR

Specificity (True Negative Rate) = $\frac{TN}{TN + FP}$ = 1 - FPR

FPR (False Positivity Rate) = $\frac{FP}{FP + TN}$ = 1 - Specificity

FNR (False Negative Rate) = $\frac{FN}{FN + TP}$ = 1 - Recall

NPV (Negative Predictive Value) = $\frac{TN}{TN + FN}$

F1 Score = 2 * $\frac{Precison * Recall}{Precision + Recall}$

Euclidiana: np.sqrt(np.sum((a - b) ** 2))

Manhattan: np.sum(np.abs(a - b))

Mahalanobis: delta = a - b
    inv_cov_matrix = inv(covariance_matrix)
    return np.sqrt(np.dot(np.dot(delta.T, inv_cov_matrix), delta))

Cossenos: 1 - (np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b)))

In [10]:
TN = cm[0][0]
FP = cm[0][1]
FN = cm[1][0]
TP = cm[1][1]

# taxa de falso positivo
tfp = FP / (FP + TN)

# taxa de verdadeiro negativo
tvn = TN / (FP + TN)

print('Falsos alarmes :', tfp)
print('Verdadeiros negativos :', 1 - tfp)
print('Verdadeiros negativos :', tvn)

Falsos alarmes : 0.18125
Verdadeiros negativos : 0.81875
Verdadeiros negativos : 0.81875


In [11]:
print(FN)
print(TN)

60
786


In [12]:
# percentagem de exemplos da classe 1
print('Percentagem de exemplos da classe 1 :', np.sum(y==1)/y.shape[0])
# recall
print('Recall :', TP/(TP+FN))

Percentagem de exemplos da classe 1 : 0.2
Recall : 0.75


In [13]:
X = np.array([[-2,1,-2,-1,3,5,3,3],
             [-2,-4,-3,-2,-2,-3,-5,-1]])
y = np.array([0,0,0,0,1,1,1,1])

In [14]:
m0 = np.mean(X[:, y==0], axis=1)
m1 = np.mean(X[:, y==1], axis=1)

mDist = spd.cityblock(m0, m1)
cDist = spd.cosine(m0, m1)

print('Distancia de Manhattan entre as medias : ', mDist)
print('Distancia de Cosseno entre as medias : ', cDist)

Distancia de Manhattan entre as medias :  4.5
Distancia de Cosseno entre as medias :  0.688094206275401


In [15]:
x2 = np.array([[-3,0],
               [4,2]])
nc = NearestCentroid(metric='manhattan').fit(X.T, y)
y2e = nc.predict(x2.T)
y2e

array([0, 0])

In [16]:
nc = NearestCentroid(metric='euclidean').fit(X.T, y)
y2e = nc.predict(X.T)
confusion_matrix(y, y2e)

array([[4, 0],
       [0, 4]])

In [17]:
x2 = np.array([[-2,3],
              [-1,1]])
nc = NearestCentroid(metric='manhattan').fit(X.T, y)
y2e = nc.predict(x2.T)
y2e

array([0, 1])

In [18]:
Xd = np.array([ [-2, 3],
                [-1, 1] ])

S0 = np.cov(X[:, y==0])
S1 = np.cov(X[:, y==1])

S0i = pinv(S0)
S1i = pinv(S1)

D0 = np.sqrt(np.sum( (Xd-m0[:, np.newaxis])*np.dot(S0i, Xd-m0[:, np.newaxis]), axis=0 ) )
D1 = np.sqrt(np.sum( (Xd-m1[:, np.newaxis])*np.dot(S1i, Xd-m1[:, np.newaxis]), axis=0 ) )

D = np.vstack((D0, D1))
print(D, '\n')
ye = np.argmin(D, axis=0)
print(ye)

[[2.06155281 9.34612219]
 [5.5218099  2.21446295]] 

[0 1]


In [19]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim_D0 = cosine_similarity(X.T, m0.reshape(1, -1))
cosine_sim_D1 = cosine_similarity(X.T, m1.reshape(1, -1))

D_cosine = np.vstack((cosine_sim_D0.T, cosine_sim_D1.T))
ye_cosine = np.argmax(D_cosine, axis=0)

print("Matriz de Similaridade (D_cosine):\n", D_cosine)
print("\nClassificação (ye_cosine):\n", ye_cosine)

Matriz de Similaridade (D_cosine):
 [[ 0.90618314  0.82884867  0.97152034  0.99340894  0.23695618  0.19047747
   0.63004086 -0.02701716]
 [-0.11914522  0.79008515  0.07788767  0.20094414  0.99696214  0.99212898
   0.93433506  0.9413394 ]]

Classificação (ye_cosine):
 [0 0 0 0 1 1 1 1]


# --------------------------------------------

In [20]:
D1 = pickle.load(open('A47467_Q001_data.p', 'rb'))
D2 = pickle.load(open('A47467_Q002_data.p', 'rb'))

In [25]:
y = D2['trueClass']
ye = D2['estClass']
confusion_matrix(y, ye)

array([[786, 174],
       [ 60, 180]])

In [26]:
TN = cm[0][0]
FP = cm[0][1]
FN = cm[1][0]
TP = cm[1][1]

print(FP / (FP + TN))
print(TP / (TP + FN))

0.18125
0.75


In [31]:
print(np.round((1 - (TP + TN)/(TP + TN + FP + FN))*100,1))
# percentagem de exemplos da classe dos positivos
print('Percentagem de exemplos da classe 1 :', np.sum(y==1)/y.shape[0])

19.5
Percentagem de exemplos da classe 1 : 0.2
